### Decision Trees
Decision trees are a powerful and commonly used machine learning technique. The basic concept is very similar to trees you may have seen commonly used to aid decision-making. Here's an example:<br>
<img src="http://localhost:8889/files/python_prog/ML/img/tree_0.png", width = 350/>
Trees can be used for classification or regression problems. In this lesson, we'll walk through the building blocks of making a decision tree automatically.<br>

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't. In our bear wrestling example, a decision tree can pick up on the fact that you should only run away from large bears when escape is impossible, whereas a linear regression would have had to weight both factors in the absence of the other.<br><br>
We'll be looking at individual income in the United States. The data is from the 1994 Census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is if they make less than or equal to 50k a year, or more than 50k a year.

In [38]:
import json
import matplotlib
import warnings
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
from IPython.display import Image


warnings.simplefilter("ignore")
root = r"/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML/styles/bmh_matplotlibrc.json"
s = json.load(open(root))
matplotlib.rcParams.update(s)
% matplotlib inline

In [29]:
def convert_to_categorical(label):
    """
    Convenience function:
        convert to pd.Categorical 
    """
    col = pd.Categorical.from_array(income[label])
    income[label] = col.codes
    
    
def compute_entropy(column):
    """
    Convenience function:
        Calculate entropy given a pandas Series, list, or numpy array
    """
    counts = np.bincount(column)
    probabilities = counts / len(column)
    
    entropy = 0
    
    for prob in probabilities:
        if prob > 0 : 
            entropy += prob * math.log(prob, 2)
        
    return -entropy

def compute_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a dataset, column to split on, and target
    """
    total_entropy = compute_entropy(data[target_name])
    
    column = data[split_name]
    median = column.median()
    
    left_split = data[column <= median]
    right_split = data[column > median]
    
    to_subtract = 0
    
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0])
        to_subtract += prob * compute_entropy(subset[target_name])
        
    return total_entropy - to_subtract

### Dataset
1. <b>Read the data</b>
2. <b>Convert categorical variables</b>

In [3]:
headers =['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
         'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
         'high_income']
income = pd.read_csv("/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML/data/adult.csv", names = headers,
                    index_col = False)
# set index_col = False to avoid pandas thinking the first column (age) is row indexes
income.tail()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [4]:
convert_to_categorical("workclass")
convert_to_categorical("education")
convert_to_categorical("marital_status")
convert_to_categorical("occupation")
convert_to_categorical("relationship")
convert_to_categorical("race")
convert_to_categorical("sex")
convert_to_categorical("native_country")
convert_to_categorical("high_income")

### Split points
A decision tree is constructed through a series of $nodes$ and $branches$. A node is where we split the data based on a variable, and a branch is one side of the split. example : <br>
<img src="http://localhost:8889/files/python_prog/ML/img/tree_1.png", width = 350/><br>
In the above diagram, the node is splitting the data based on if the individual works in the private sector. There are two branches, No, and Yes. The workclass column indicates the sector people work in. The value Private in that column has been mapped to the numeric code 4 (we can check this by comparing the values in the workclass column that used to be labelled Private with the current values to see where they line up). So the No branch corresponds to workclass != 4, and the Yes branch corresponds to workclass == 4.

This is exactly how a decision tree works -- we keep splitting the data based on variables. As we do this, the tree gets deeper and deeper. The tree we made above is two levels deep, because it has one split, and two "levels" of nodes.

The tree below is 3 levels deep.
<img src="/files/python_prog/ML/img/tree_2.png", width = 350/>

In [9]:
privates_incomes = income[income.workclass ==  4]
public_incomes = income[income.workclass != 4]
print(len(privates_incomes))
print(len(public_incomes))

22696
9865


<i>When we performed the split, 9865 rows went to the left, where workclass does not equal 4, and 22696 rows went to the right, where workclass equals 4.<br>

It's useful to think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some rows will go to the left. As we build the tree deeper and deeper, each node will "receive" less and less rows.<br>

Here's a look at the splits, and the number of rows that will exist at each node:
<img src="/files/python_prog/ML/img/tree_3.png", width = 350/>


### Rationale behind splitting
The nodes at the bottom of the tree, when we decide to stop splitting, are called <b><i>terminal nodes</i></b>, or <i><b>leaves</b><i/>. When we do our splits, we aren't doing them randomly -- we have an objective. Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

We're trying to predict the <b>high_income</b> column.
If high_income is 1, it means that the person has an income higher than 50k a year.
If high_income is 0, it means that they have an income less than or equal to 50k a year.

After constructing a tree using the income data, we'll want to make predictions. In order to do this, we'll take a new row, and feed it through our decision tree. We'll check to see if the person works in the private sector. If they do, we'll check to see if they are native to the US, and so on.

We'll eventually reach a leaf. The leaf will tell us what value we should predict for high_income.

In order to be able to make this prediction, all leaves need to have rows with only a single value for high_income. So leaves can't have both 0 and 1 values in the high_income column. Each leaf can only have rows with the same values for our target. If this doesn't happen, we won't be able to make effective predictions.

So, we keep splitting nodes until we get to a point where all the rows in a node have the same value for high_income.

### Entropy
Now that we have a high-level view of how decision trees work, let's get into the details and figure out how to perform the splits.

In order to do this, we'll use a measure to figure out which variable to split a node on. Post-split, we'll have two datasets, each containing the rows from one side of the split.

Since we're trying to get to leaves consisting of only 1s or only 0s for high_income, each split will need to get us closer to that goal.

When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the high_income column are.

A commonly used metric is called entropy. Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A dataset consisting of only 1s in the high_income column would have low entropy.

Entropy, which is not to be confused with entropy from physics, comes from information theory. Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. A key concept in this is the notion of a bit of information. One bit of information is one unit of information. It can be represented as a binary number -- a bit of information either has the value 1, or 0. If there's an equal probability of tomorrow being sunny (1) and tomorrow being not sunny (0), if I tell you that it will be be sunny, I've given you one bit of information.

Entropy can also be thought of in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. Thus, we gain no new information by flipping the coin, so entropy is 0. On the other hand, if the coin has a heads side and a tails side, there's a 50% probability of landing on either. Thus, flipping the coin gives us one bit of information -- which side the coin landed on. There's much more complexity, especially when you get to cases with more than two possible outcomes, or differential probabilities. However, a deep understanding of entropy isn't necessary for constructing decision trees.


The formula for $entropy$ is :
$$- \sum_{i=1}^c P(x_i)\log_{b}(P(x_i))$$

example : 
<img src="http://localhost:8889/files/python_prog/ML/img/tree_4.png", width = 150/>
Entropy = $-((2 / 5 * \log_{2}(2/5)) + (3/5 * \log_{2}(3/5))) = .97$

We get less than one "bit" of information -- we only get .97. This is because there are slightly more 1s in the sample data than 0s. This means that if we were predicting a new value, we could guess that the answer is 1, and be right more often than we're wrong (there's a .6 probability of the answer being 1). Because of this prior knowledge, we gain less than a full "bit" of information when we observe a new value.

In [37]:
# Entropy of the high_income
prob_0 = len(income[income.high_income == 0])/len(income.high_income)
prob_1 = len(income[income.high_income == 1])/len(income.high_income)

income_entropy = -(prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1, 2))
print(income_entropy)

0.7963839552022132


### Information Gain
We'll need a way to go from computing entropy to figuring out which variable to split based on. We can do this using <b><i>information gain</i></b>.<br> Information gain tells us which split will reduce entropy the most.

Here's the formula for information gain:
$$IG(T, A) = Entropy(T) - \sum_{v\in A}\frac{T_{v}}{T} . Entropy(T_v)$$

This may look complicated, but we'll break it down. We're computing information gain $(IG)$ for a given target variable ($T$), and a given variable that we want to split based on ($A$). To compute this, we first calculate the entropy for T. Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. We then multiply this by the entropy of the rows where A is v.
We add together all of these subset entropies, and subtract from the overall entropy to get information gain.

For an even simpler explanation, we're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If it's positive, we've lowered entropy with our split. The higher it is, the more we've lowered entropy.

One strategy to construct trees is to create as many branches at each node as there are unique values for the variable being split on. So if the variable has 3 or 4 values, that would result in 3 or 4 branches. Usually, this is more complexity than it's worth, and doesn't improve prediction accuracy, but it's worth knowing about.

To simplify the calculation of information gain, and to make splits simpler, we won't do it for each unique value. Instead, for the variable we're splitting on, we'll find the median. Any rows where the value of the variable is below the median will split left, and the rest of the rows will split right. To compute information gain, we'll only have to compute entropies for two subsets.

In [30]:
# compute the IG for splitting on the age column of income : 
median_age = income.age.median()
left_split = income[income.age <= median_age]
right_split = income[income.age > median_age]

age_information_gain = income_entropy - \
(((left_split.shape[0] / income.shape[0]) * compute_entropy(left_split["high_income"])) + \
(right_split.shape[0] / income.shape[0]) * compute_entropy(right_split["high_income"]))

print(age_information_gain)

0.0470286613047


### Finding The Best Split
Now that we know how to compute information gain, we can compute the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by finding which split would have the highest information gain.

In [36]:
_headers =['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
         'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']
information_gain = {}
for col_name in _headers : 
    information_gain[col_name] = compute_information_gain(income, col_name, "high_income")
    
print(max(information_gain, key = information_gain.get))

marital_status


So far, we've been following the <b>$ID3$ $algorithm$</b> to construct <i>decision trees</i>. There are other algorithms, including <b>$CART$</b> that use other measures for the split criterion. We'll learn more about these other algorithms in future.

Next, we'll figure out how to recursively construct an entire tree, using the ID3 algorithm.